In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
from torch.autograd import Variable

In [16]:
import numpy as np
store = np.load('../data/processed_eeg.npy')
store.shape

(113, 120, 4)

In [17]:
# store = np.repeat(store, 2, axis = 0)

In [18]:
df = pd.read_csv('../data/df_onsite.csv')
lvo = df['lvo'].to_numpy()
lvo = np.delete(lvo, 87)
lvo = lvo.reshape(lvo.shape[0],-1)
lvo.shape


(113, 1)

In [19]:
lvo = np.repeat(lvo, 4, axis=0)

In [20]:
print(store.shape)
print(lvo.shape)

(113, 120, 4)
(452, 1)


In [21]:
X_train_tensors = Variable(torch.Tensor(store))
# X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(lvo))
# y_test_tensors = Variable(torch.Tensor(y_test)) 

In [22]:
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 120, X_train_tensors.shape[2]))


# X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

In [23]:
X_train_tensors_final.size()

torch.Size([113, 120, 4])

In [24]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out
        

In [25]:
num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 4 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 4 #number of stacked lstm layers

num_classes = 1 #number of output classes 

In [26]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]) #our lstm class 


In [27]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 

In [28]:
for epoch in range(num_epochs):
  outputs = lstm1.forward(X_train_tensors_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.25975
Epoch: 100, loss: 0.21408
Epoch: 200, loss: 0.20706
Epoch: 300, loss: 0.20441
Epoch: 400, loss: 0.20142
Epoch: 500, loss: 0.19942
Epoch: 600, loss: 0.19799
Epoch: 700, loss: 0.19514
Epoch: 800, loss: 0.19179
Epoch: 900, loss: 0.18987
